In [ ]:
!pip install torch_geometric
!pip install captum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.4 MB/s eta 0:00:00


In [ ]:
from google.colab import files
files.upload()

#jupytext --to ipynb Comparative_Cheb_GCN.py
# Don't forget to upload model as well
!unzip numpy_data.zip

Saving numpy_data.zip to numpy_data.zip
Saving model_342_finishtraininglater.pt to model_342_finishtraininglater.pt
Archive:  numpy_data.zip
  inflating: AdjacencyMatrix.npy     
  inflating: new_data.npy            
  inflating: new_labels.npy          
  inflating: NodeIndex.npy           


In [ ]:
import time
start_time = time.time()

from torch_geometric.data import Data, DataLoader
from torch_geometric.datasets import TUDataset, Planetoid
from torch_geometric.nn import GCNConv, Set2Set
from torch_geometric.explain import GNNExplainer
import torch_geometric.transforms as T
import torch
#import torch.nn.functional as F
import os
from tqdm import tqdm, trange
#
import matplotlib.pyplot as plt

#Loading saved model architecture
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import ChebConv
from torch_geometric.nn import GATConv
from torch_geometric.nn import TransformerConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn.pool import global_max_pool

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)

        self.conv1 = ChebConv(6, 512, K = 3)
        self.conv2 = ChebConv(512,512, K = 4)
        self.conv3 = ChebConv(512, 512, K = 5)

        self.lin = Linear(512, 187)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        x = self.conv1(x, edge_index)
        x = x.relu()

        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        # x = global_max_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training) #0.5
        x = self.lin(x)

        return x

model = GCN(hidden_channels=464)
print(model)

print(os.getcwd()+ "/model_342_finishtraininglater.pt")
model_save_location = os.getcwd()+ "/model_342_finishtraininglater.pt"
model.load_state_dict(torch.load(model_save_location, map_location=torch.device('cpu')))
# model.load_state_dict(torch.load(model_save_location))
print(model.conv3)
print(model.state_dict())


import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import os
import torch
import networkx as nx
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
print(os.getcwd())

# +
import time
#Extract Training Data
#Loads IEEE34 Bus Simulation Data

path = os.getcwd()

numpy_data = np.load(path+"/new_data.npy",allow_pickle = True)
numpy_data = np.squeeze(numpy_data)
print(numpy_data)
print(numpy_data.shape)
print(numpy_data.size)

value_data = np.load(path+"/new_labels.npy",allow_pickle = True)
print(value_data)
print(value_data.shape)
print(value_data.size)


NodeIndex = np.load(path+"/NodeIndex.npy",allow_pickle = True)
print(NodeIndex)
print(NodeIndex.shape)
print(NodeIndex.size)

AdjacencyMatrix = np.load(path+"/AdjacencyMatrix.npy",allow_pickle = True)
print(AdjacencyMatrix)
print(AdjacencyMatrix.shape)
print(AdjacencyMatrix.size)

# +
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import NormalizeFeatures
from torch.nn.functional import normalize


encoder = ['P1', 'P2', 'P3', 'P7', 'P6', 'P11', 'P27', 'P44', 'P45', 'P62', 'P10',
        'P81', 'P97', 'P116', 'P117', 'P134', 'P12', 'P13', 'P14', 'P15', 'P16',
        'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P28',
        'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35', 'P36', 'P37', 'P38', 'P39',
        'P40', 'P41', 'P42', 'P43', 'P46', 'P47', 'P48', 'P49', 'P50', 'P51', 'P52',
        'P53', 'P54', 'P55', 'P56', 'P57', 'P58', 'P59', 'P60', 'P61', 'P63', 'P64',
        'P65', 'P66', 'P67', 'P68', 'P69', 'P70', 'P71', 'P72', 'P73', 'P74', 'P75',
        'P76', 'P77', 'P78', 'P79', 'P80', 'P82', 'P83', 'P84', 'P85', 'P86', 'P87',
        'P88', 'P89', 'P90', 'P91', 'P92', 'P93', 'P94', 'P95', 'P96', 'P98', 'P99',
        'P100', 'P101', 'P102', 'P103', 'P104', 'P105', 'P106', 'P107', 'P108', 'P109',
        'P110', 'P111', 'P112', 'P113', 'P114', 'P115', 'P118', 'P119', 'P120', 'P121',
        'P122', 'P123', 'P124', 'P125', 'P126', 'P127', 'P128', 'P129', 'P130', 'P131',
        'P132', 'P133', 'P135', 'P136', 'P137', 'P138', 'P139', 'P140', 'P141', 'P142',
        'P143', 'P144', 'P145', 'P146', 'P147', 'P148', 'P149', 'P150', 'S194', 'S193',
        'S195', 'S199', 'S198', 'S200', 'S204', 'S203', 'S205', 'S206', 'S211', 'S210',
        'S212', 'S213', 'S218', 'S217', 'S219', 'S220', 'S225', 'S224', 'S226', 'S227',
        'S232', 'S231', 'S233', 'S237', 'S236', 'S239', 'S1', 'S2', 'S3', 'S4', 'S5',
        'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17',
        'S18', 'S19', 'S20', 'S21', 'S42', 'S43', 'S44', 'S45', 'S46', 'S47', 'S48',
        'S49', 'S50', 'S51', 'S52', 'S53', 'S54', 'S55', 'S56', 'S57', 'S58', 'S59',
        'S60', 'S61', 'S62', 'S83', 'S84', 'S85', 'S86', 'S87', 'S88', 'S89', 'S90',
        'S91', 'S92', 'S93', 'S94', 'S95', 'S96', 'S97', 'S98', 'S99', 'S100', 'S101',
        'S102', 'S103', 'S124', 'S125', 'S126', 'S127', 'S128', 'S129', 'S130', 'S131',
        'S132', 'S133', 'S134', 'S135', 'S136', 'S137', 'S138', 'S139', 'S140', 'S141',
        'S142', 'S143', 'S144', 'S22', 'S27', 'S32', 'S37', 'S63', 'S68', 'S73', 'S78',
        'S104', 'S109', 'S114', 'S119', 'S23', 'S28', 'S33', 'S38', 'S64', 'S69', 'S74',
        'S79', 'S105', 'S110', 'S115', 'S120', 'S24', 'S29', 'S34', 'S39', 'S65', 'S70',
        'S75', 'S80', 'S106', 'S111', 'S116', 'S121', 'S25', 'S30', 'S35', 'S40', 'S66',
        'S71', 'S76', 'S81', 'S107', 'S112', 'S117', 'S122', 'S26', 'S31', 'S36', 'S41',
        'S67', 'S72', 'S77', 'S82', 'S108', 'S113', 'S118', 'S123', 'P4', 'P5', 'P8',
        'P9', 'S148', 'S145', 'S152', 'S149', 'S156', 'S153', 'S160', 'S157', 'S147',
        'S146', 'S151', 'S150', 'S155', 'S154', 'S159', 'S158', 'S164', 'S161', 'S168',
        'S165', 'S172', 'S169', 'S176', 'S173', 'S163', 'S162', 'S167', 'S166', 'S171',
        'S170', 'S175', 'S174', 'S180', 'S177', 'S184', 'S181', 'S188', 'S185', 'S192',
        'S189', 'S179', 'S178', 'S183', 'S182', 'S187', 'S186', 'S191', 'S190', 'S196',
        'S197', 'S201', 'S202', 'S207', 'S208', 'S209', 'S214', 'S215', 'S216', 'S221',
        'S222', 'S223', 'S228', 'S229', 'S230', 'S234', 'S235', 'S238', 'S240']

print(encoder)

research_paper_decoder = [0, 1, 2, 3, 2, 4, 5, 6, 7, 8, 3, 9, 10, 11, 12, 13, 4, 14, 15, 14, 16,
                          17, 14, 18, 14, 19, 20, 14, 21, 14, 22, 5, 14, 23, 14, 24, 14, 25, 14,
                          26, 14, 27, 14, 28, 29, 14, 30, 7, 14, 31, 14, 32, 14, 33, 14, 34, 35,
                          36, 14, 37, 14, 38, 39, 8, 14, 40, 14, 41, 14, 42, 14, 43, 14, 44, 45,
                          14, 46, 14, 47, 14, 48, 9, 14, 49, 14, 50, 51, 14, 52, 14, 53, 54, 14,
                          55, 14, 56, 10, 14, 57, 14, 58, 14, 59, 14, 60, 61, 14, 62, 14, 63, 14,
                          64, 14, 65, 12, 14, 66, 14, 67, 14, 68, 14, 69, 14, 70, 14, 71, 72, 14,
                          73, 13, 14, 74, 14, 75, 76, 14, 77, 14, 78, 79, 80, 14, 81, 14, 82, 29,
                          83, 63, 39, 84, 72, 20, 85, 45, 78, 46, 86, 54, 80, 25, 87, 35, 61, 34,
                          88, 59, 68, 42, 89, 75, 16, 90, 51, 91, 92, 93, 73, 94, 95, 96, 97, 56,
                          98, 99, 100, 101, 38, 102, 103, 104, 105, 22, 106, 107, 108, 109, 55, 70,
                          110, 111, 112, 81, 44, 113, 114, 115, 21, 79, 116, 117, 118, 47, 27, 119,
                          120, 121, 122, 36, 74, 123, 124, 125, 77, 32, 126, 127, 128, 18, 58, 129,
                          130, 131, 33, 76, 132, 133, 134, 135, 40, 136, 137, 138, 139, 31, 140,
                          141, 142, 143, 23, 144, 145, 146, 147, 24, 148, 149, 150, 151, 82, 152,
                          153, 154, 62, 155, 156, 157, 17, 158, 159, 160, 65, 64, 161, 162, 19, 69,
                          163, 164, 67, 66, 165, 166, 48, 71, 167, 168, 53, 60, 169, 170, 50, 57,
                          171, 172, 30, 28, 173, 174, 52, 26, 175, 176, 15, 41, 177, 178, 179, 37,
                          180, 181, 182, 43, 183, 184, 185, 49, 186, 2, 2, 3, 3, 82, 73, 65, 56,
                          48, 38, 30, 22, 55, 64, 81, 71, 21, 28, 47, 37, 62, 70, 19, 44, 53, 79,
                          52, 27, 36, 69, 77, 60, 18, 26, 33, 43, 17, 74, 67, 32, 50, 58, 15, 76,
                          40, 66, 31, 57, 23, 41, 24, 49, 29, 63, 39, 72, 20, 45, 78, 46, 54, 80,
                          25, 35, 61, 34, 59, 68, 42, 75, 16, 51]

print(research_paper_decoder)
print(len(research_paper_decoder))


FaultLocationLabels = value_data[:,3]

for n in range(len(FaultLocationLabels)):
    FaultLocationLabels[n]=research_paper_decoder[encoder.index(str(FaultLocationLabels[n]).upper())]



y = FaultLocationLabels.astype("int64")
y = torch.from_numpy(y)
x = numpy_data.astype("float32")
x = torch.from_numpy(x)

NodeIndex = NodeIndex.astype("int64")
NodeIndex = NodeIndex.T
NodeIndex = torch.from_numpy(NodeIndex)

print(y)
print(y.dtype)
print(x)
print(x.dtype)
print(NodeIndex)
print(NodeIndex.dtype)
#
print(x[0])
print(normalize(x[0]))
#
result_translator = np.unique(FaultLocationLabels.astype("int64")).tolist()
print()
total_data_list = []
for n in range(len(x)):
    #print(x[n])
    #print(y[n])
    DataObject = Data(x = x[n], edge_index = NodeIndex, y = y[n], is_undirected = True) #Testing with non-normalized data
    #DataObject = Data(x = x[n], edge_index = NodeIndex, y = y[n], is_undirected = True)
    DataObject.is_undirected = True
    total_data_list.append(DataObject)
#print('Y'*888)
#print(total_data_list[0].x)

print()
#print(f'Dataset: {total_data_list}:')
print('===================')
print(f'Number of graphs: {len(total_data_list)}')
print(f'Number of features: {total_data_list[0].num_features}')
#print(f'Number of classes: {total_data_list[0].num_classes}')

data = total_data_list[0]  # Get the first graph object.
#print(data)
#print(data.y)

print()
print(NormalizeFeatures(data.x))
print(data.x)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

from random import shuffle
torch.manual_seed(12345)
#total_data_list = total_data_list.shuffle()
shuffle(total_data_list)

train_dataset = total_data_list[:24960] #9150 is half
test_dataset = total_data_list[24960:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')


print(test_dataset[0].x)
print(test_dataset[0])
#print(train_dataset[0].x)

for sample in range(len(test_dataset)):
    # noise = np.random.normal(1,0.03, size = (37,6)) #0.09 #Uncomment for noise
    # noise = noise.astype("float32")
    # noise = torch.from_numpy(noise)
    # test_dataset[sample].x = noise*test_dataset[sample].x
    test_dataset[sample].x = normalize(test_dataset[sample].x)
    #print(sample)

print('Y'*888)
#print(train_dataset[0].x)
print(test_dataset[0].x)
print(test_dataset[0])
##noise = np.random.normal(1,0.09, size = (16,6))
##noise = noise.astype("float32")
##noise = torch.from_numpy(noise)
print('=============================================================')
##
print(train_dataset[0].x)
print(train_dataset[0])
#print(train_dataset[0].x)

for sample in range(len(train_dataset)):
#    noise = np.random.normal(1,0.09, size = (16,6))
#    #    noise = np.random.normal(1,0.09, size = (1,96))
#    noise = noise.astype("float32")
#    noise = torch.from_numpy(noise)
##    print(noise)
##    X_test[sample] = X_test[sample]*noise[0]
#    train_dataset[sample].x = noise*train_dataset[sample].x
    train_dataset[sample].x = normalize(train_dataset[sample].x)
    #print(sample)

print('Y'*888)
#print(train_dataset[0].x)
print(train_dataset[0].x)
print(train_dataset[0])
##print(noise*test_dataset[0].x)
# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

print("$"*200)

#################################################################################
import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import torch_geometric.transforms as T
from torch_geometric.datasets import ExplainerDataset
from torch_geometric.datasets.graph_generator import BAGraph
from torch_geometric.explain import Explainer, GNNExplainer, GraphMaskExplainer
from torch_geometric.explain import DummyExplainer, PGExplainer
from torch_geometric.nn import GCN
from torch_geometric.utils import k_hop_subgraph


##dataset = ExplainerDataset(
##    graph_generator=BAGraph(num_nodes=300, num_edges=5),
##    motif_generator='house',
##    num_motifs=80,
##    transform=T.Constant(),
##)
#data = test_dataset[0]
#print(data)

from typing import Tuple

import torch
from torch import Tensor

from torch_geometric.explain import Explainer, Explanation
from torch_geometric.explain.config import ExplanationType, ModelMode


pos_fidelity_list = []
neg_fidelity_list = []
characterization_list = []
# topk_list = [10,50,100,150,200,300]
topk_list = [300]
# topk_list = [1,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80]
for topk in topk_list:
    explainer = Explainer(
        model=model,
    #    algorithm=GNNExplainer(epochs=200),
    #    algorithm=PGExplainer(epochs=2),
        # algorithm=GraphMaskExplainer(num_layers = 3, epochs=200),
       algorithm=DummyExplainer(),
        explanation_type='model',
        node_mask_type='attributes',
        edge_mask_type='object',
        model_config=dict(
            mode='multiclass_classification',
            task_level='graph',
            return_type='log_probs',
        ),
        threshold_config=dict(
#            threshold_type='topk',
            threshold_type='topk_hard',
            value =topk,
        ),
    )
    count = 0
    explain_y_hat_list = []
    complement_y_hat_list = []
    y_list = []
    y_hat_list = []

    for batch in test_loader:
        count+=1
        Edge_Index = batch.edge_index[:,:780]
        Batch = batch.batch[:390]
        label1 = batch.y[0].numpy().tolist()#
        input = batch.x[0:390]#
        y_hat = model(input, Edge_Index, Batch).argmax(dim=1).item()

        explanation = explainer(x = input, edge_index = Edge_Index, batch = Batch)#, target = label1
        # y_hat = model(input, Edge_Index, Batch).argmax(dim=1).item()
        node_mask = explanation.get('node_mask')
        edge_mask = explanation.get('edge_mask')

        #Two lines below focus on converting the soft masks to hard masks
     #   node_mask = torch.tensor((node_mask-node_mask.mean())>0, dtype=torch.float32)
    #    edge_mask = torch.tensor((edge_mask-edge_mask.mean())>0, dtype=torch.float32)
        kwargs = {key: explanation[key] for key in explanation._model_args}
    #    y = explanation.target
        y = label1
        explain_y_hat = explainer.get_masked_prediction(
                explanation.x,
                explanation.edge_index,
                node_mask,
                edge_mask,
                **kwargs,
            )
        explain_y_hat = explainer.get_target(explain_y_hat)
        complement_y_hat = explainer.get_masked_prediction(
            explanation.x,
            explanation.edge_index,
            1. - node_mask if node_mask is not None else None,
            1. - edge_mask if edge_mask is not None else None,
            **kwargs,
        )
        complement_y_hat = explainer.get_target(complement_y_hat)

        explain_y_hat_list.append(explain_y_hat.numpy()[0])
        complement_y_hat_list.append(complement_y_hat.numpy()[0])
        y_list.append(y)
        y_hat_list.append(y_hat)

        print(count)
        print(explain_y_hat.numpy()[0])
        print(complement_y_hat.numpy()[0])
        print(y)
        print(y_hat)
        print("#########")

#        if count > 1:
#            break

    explain_y_hat_list = np.array(explain_y_hat_list)
    complement_y_hat_list = np.array(complement_y_hat_list)
    y_list = np.array(y_list)
    y_hat_list = np.array(y_hat_list)

    print(explain_y_hat_list )
    print(complement_y_hat_list )
    print(y_list )
    print(y_hat_list)
    #.astype('float').mean()

    pos_fidelity = 1. - (complement_y_hat_list == y_hat_list).astype('float').mean()
    neg_fidelity = 1. - (explain_y_hat_list == y_hat_list).astype('float').mean()
    print("Positive Fidelity = ", pos_fidelity) #positive fidelity reflects necessary explanation
    # It's better for positive fidelity to be 1
    print("Negative Fidelity = ", neg_fidelity) #negative fidelity reflects sufficient explanation
    # It's better for negative fidelity to be 0
    CharacterizationScore = 1./((0.5/pos_fidelity)+(0.5/(1-neg_fidelity)))
    print(CharacterizationScore)

    pos_fidelity_list.append(pos_fidelity)
    neg_fidelity_list.append(neg_fidelity)
    characterization_list.append(CharacterizationScore)

print(topk_list)
print(pos_fidelity_list)
print(neg_fidelity_list)
print(characterization_list)

Topk_Results = pd.DataFrame(data = {'TopK' : topk_list,
    'Fidelity+' : pos_fidelity_list,
    'Fidelity-' : neg_fidelity_list,
    'Characterization' : characterization_list})

#Topk_Results.to_csv(path+'/'+str(explainer.algorithm)[:-2]+'_topk_results.csv', index = False)
Topk_Results.to_csv(path+'/'+str(explainer.algorithm)[:-2]+'_topk_hard_results.csv', index = False)
#.attribution_method_class.__name__
print("  "+str(explainer.algorithm)[:-2])
#print(type(explainer.algorithm.attribution_method_class))
print(path+'/'+str(explainer.algorithm)[:-2]+'_topk_results.csv')
print(Topk_Results)

end_time = time.time()
print("Run Time = ", end_time-start_time)

print(test_loader)



GCN(
  (conv1): ChebConv(6, 512, K=3, normalization=sym)
  (conv2): ChebConv(512, 512, K=4, normalization=sym)
  (conv3): ChebConv(512, 512, K=5, normalization=sym)
  (lin): Linear(in_features=512, out_features=187, bias=True)
)
/content/model_342_finishtraininglater.pt
ChebConv(512, 512, K=5, normalization=sym)


<ipython-input-3-049a3ec3fb0a>:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_location, map_location=torch.device('cpu')))


Streaming output truncated to the last 5000 lines.
77
77
#########
5411
14
14
143
143
#########
5412
28
14
4
4
#########
5413
14
14
16
14
#########
5414
14
14
24
24
#########
5415
14
14
4
5
#########
5416
14
14
158
158
#########
5417
14
14
5
5
#########
5418
14
14
52
52
#########
5419
14
14
48
48
#########
5420
28
14
4
4
#########
5421
14
14
45
20
#########
5422
14
14
149
149
#########
5423
14
14
112
129
#########
5424
14
14
14
14
#########
5425
14
14
171
171
#########
5426
14
14
71
71
#########
5427
14
14
20
45
#########
5428
14
14
172
172
#########
5429
14
14
34
34
#########
5430
14
14
36
36
#########
5431
14
14
31
31
#########
5432
28
14
2
2
#########
5433
14
14
14
14
#########
5434
14
14
153
153
#########
5435
14
14
143
143
#########
5436
14
14
39
39
#########
5437
14
14
34
34
#########
5438
14
14
173
173
#########
5439
14
14
24
24
#########
5440
14
14
9
9
#########
5441
14
14
14
14
#########
5442
14
14
13
13
#########
5443
14
14
46
46
#########
5444
14
14
52
52
#########
5445
14
1